# CTBI Neurobot databases merger and downloader for WP15
By Stephen Larroque @ Coma Science Group, GIGA Research, University of Liege
Creation date: 2018-10-27
License: MIT
v0.2.0

DESCRIPTION:
Generic tool to merge two neurobot CSV databases based on GUPI

INSTALL NOTE:
You need to pip install pandas before launching this script.
Tested on Python 2.7.13

USAGE: For the merge: First download from Neurobot the Imaging (and Subject) database in one CSV, then the Outcomes database in another CSV. Then you need to convert the comma separated csv files to semicolon (or change all the pd.read_csv() calls in this notebook to use commas instead of semicolon). Then you can input the path of these csv files below and run all cells!
For the imaging nifti download: you need to input your credentials in login.cfg.example and rename to login.cfg. You also need the Imaging/Subject databases in a csv.

TODO:


In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
# AUX FUNCTIONS

import os, sys

cur_path = os.path.realpath('.')
sys.path.append(os.path.join(cur_path, 'csg_fileutil_libs'))  # for unidecode and cleanup_name, because it does not support relative paths (yet?)

import ast
import re
import pandas as pd

from csg_fileutil_libs.aux_funcs import save_df_as_csv, _tqdm, merge_two_df, df_remap_names, concat_vals, df_concatenate_all_but


In [ ]:
# PARAMETERS

# First (ID) database to merge (both need to have a column 'name'). The merged 'name' column will use the names from this database.
db_imaging = r'neurobot_subjects-imaging_20181027.csv'
# Second (reference) database to merge. The names will be added as a new column 'name_altx'.
db_outcome = r'neurobot_outcomes_20181027.csv'

# Output database with the merge results
out_db = r'neurobot_merged_db_20181027.csv'
# Output folder to store imaging data
out_imdir = r'F:\neurobot_mri'
# Uncompress nifti files on-the-fly?
nifti_ungz = True
# When downloading the nifti files, pass errors (but will still be printed) - else the program will stop at the first exception
pass_errors = True

verbose = False

# What kind of series (DTI, etc) to include? (look at the Imaging.ScanType column)
#series_to_include = []


## Database merging

In [ ]:
# Load first database
df_im = pd.read_csv(db_imaging, sep=';').dropna(how='all').dropna(how='any', subset=['gupi', 'Imaging.SubjectGroup'])  # drop all rows where name is empty (necessary else this will produce an error, we expect the name to exist)
df_im

In [ ]:
# Load second database
df_oc = pd.read_csv(db_outcome, sep=';').dropna(how='all').dropna(how='any', subset=['gupi'])  # drop all rows where name is empty (necessary else this will produce an error, we expect the name to exist)
df_oc

In [ ]:
# Keep only MR 2 weeks
df_im = df_im.loc[(df_im['Imaging.Timepoint'] == 'MR 2 weeks'), :]
df_im

In [ ]:
# Merge both databases if gupi matches (keep only those that are in df_im - thus have an MR 2 weeks!)
df_merge, df_final = merge_two_df(df_im, df_oc, col='gupi', mode=0, dist_threshold=0, dist_words_threshold=0, skip_sanity=True, keep_nulls=1, returnmerged=True)
df_final.set_index('gupi', inplace=True)
df_merge

In [ ]:
df_final

In [ ]:
if save_df_as_csv(df_final.reset_index(), out_db, fields_order=list(df_final.columns), csv_order_by='gupi'):
    print('Merged database successfully saved in %s!' % out_db)
else:
    print('ERROR: the merged database could not be saved!')

## Database imaging downloading

In [ ]:
import gzip
import json
import re
import requests
import shutil
from urlparse import urlparse

In [ ]:
# Loading login infos
cfgpath = os.path.join(os.getcwd(), 'login.cfg')
with open(cfgpath) as f:
    # Strip out comments first
    login_infos = f.read()
    login_infos = re.sub(r'\\\n', '', login_infos)
    login_infos = re.sub(r'//.*\n', '\n', login_infos)
    # Load as JSON
    login_infos = json.loads(login_infos)

In [ ]:
# Load first database
df_im = pd.read_csv(db_imaging, sep=';').dropna(how='all').dropna(how='any', subset=['gupi', 'Imaging.SubjectGroup'])  # drop all rows where name is empty (necessary else this will produce an error, we expect the name to exist)
df_im.set_index('gupi', inplace=True)
df_im

In [ ]:
# Keep only MR 2 weeks
df_im = df_im.loc[(df_im['Imaging.Timepoint'] == 'MR 2 weeks'), :]
df_im

In [ ]:
# Precalculate total number of files to download (to show a progressbar)
#total = 0
#for idx, row in df_final.iterrows():
#    try:
#        niftiurls = ast.literal_eval(row['Imaging.NiftiURL'])
#        total += len(niftiurls)
#    except Exception:
#        niftiurls = row['Imaging.NiftiURL']
#        total += 1
#    try:
#        dicomheadersurls = ast.literal_eval(row['Imaging.DicomHeaderURL'])
#        total += len(dicomheadersurls)
#    except Exception:
#        dicomheadersurls = row['Imaging.DicomHeaderURL']
#        total +=1
#total

In [ ]:
total = df_im['Imaging.NiftiURL'].dropna().count() + df_im['Imaging.DicomHeaderURL'].dropna().count()
total

In [ ]:
def pathsafe(s):
    """Make sure a string is path safe (replace any path unsafe character). From https://stackoverflow.com/a/295146/1121352"""
    import string
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    valid_chars = frozenset(valid_chars)
    return ''.join(c for c in s if c in valid_chars)

# Create output directory if does not exist
if not os.path.exists(out_imdir):
    os.makedirs(out_imdir)

# Prepare progressbar
pbar = _tqdm(total=total, desc="DOWN", unit="files")

# Iterate for each row/gupi/subject
for gupi, row in df_im.iterrows():
    niftiurl = row['Imaging.NiftiURL']
    dicomheaderurl = row['Imaging.DicomHeaderURL']
    if verbose:
        print(niftiurl, dicomheaderurl)

    for url in (niftiurl, dicomheaderurl):
        # Get filename from server
        filename = os.path.basename(urlparse(url).path)
        # Build local filepath according to site name and scan type (and clean up each item to make sure it's pathsafe)
        filepath = os.path.join(out_imdir, pathsafe(row['Subject.SiteCode']), pathsafe(gupi), pathsafe(row['Imaging.ScanType'].replace(' ', '_') + ' ' + row['Imaging.SeriesDescription'].replace(' ', '_')), pathsafe(filename))

        # Access online the resource
        r = requests.get(url, auth=(login_infos['username'],login_infos['password']))

        # Try to download
        try:
            if r.status_code == 200:
                # Create folder if necessary
                if not os.path.exists(os.path.dirname(filepath)):
                    os.makedirs(os.path.dirname(filepath))
                # Write the content (download)
                with open(filepath, 'wb') as out:
                    for bits in r.iter_content():
                        out.write(bits)
                # Uncompress file if user wants
                if nifti_ungz and filepath[-3:] == '.gz':
                    with gzip.open(filepath, 'rb') as f_in:
                        with open(filepath[:-3], 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)
                    os.remove(filepath)
                pbar.update()
            else:
                print('Warning: could not download file: %s of subject gupi %s, got this response code: %i' % (filename, row['gupi'], r.status_code))
        except Exception as exc:
            print(exc)
            if not pass_errors:
                raise(exc)

print('All files successfully downloaded!')